# Model building - VAR

Source for how to use statsmodels VAR model for data showing seasonality: https://stats.stackexchange.com/questions/253355/time-series-data-with-seasonality-using-var#:~:text=2%20Answers&text=VAR%20models%20are%20routinely%20used,GDP%20or%20unemployment)%20are%20seasonal.
Source for code: Mahdi Shadkham-Farokhi - General Assembly - Time Series analysis lesson.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import six
import joblib
import sys
sys.modules['sklearn.externals.six'] = six
sys.modules['sklearn.externals.joblib'] = joblib

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR

from pmdarima import auto_arima

from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
df = pd.read_csv("./data/WQ_FINAL_with_Parameters.csv", parse_dates=['Date_Time'], index_col=['Date_Time'])

In [ ]:
df.shape

In [ ]:
df.head(2)

**Note:** 
we can separate data
- by HUCNAME (use avarage parameter values by day)
- by SampleDepth (aggregate data for depths where not many sampling points)

**Separate Data by HUCNAME**

In [ ]:
# check the amount of datapoints available for each HUC station
# df[['HUCNAME_']].value_counts(normalize=True)

In [ ]:
# df.groupby(by='SampleDepth')

In [ ]:
# outlet = df.loc[(df['HUCNAME_'] == 'Outlet Potomac River')][['SampleDepth', 'Parameter_CHLA', 'Parameter_DO',
#        'Parameter_NH4F', 'Parameter_NO3F', 'Parameter_PH', 'Parameter_PO4F',
#        'Parameter_SALINITY', 'Parameter_SECCHI', 'Parameter_TALK',
#        'Parameter_TDS', 'Parameter_TKNW', 'Parameter_TN', 'Parameter_TP',
#        'Parameter_TSS', 'Parameter_TURB_NTU', 'Parameter_WTEMP']]

In [ ]:
# plt.figure(figsize=(20, 8))
# outlet.groupby(by='SampleDepth')['Parameter_CHLA'].count().sort_values(ascending=False).plot(kind='bar');
# plt.ylim(bottom=100);

**Select parameter columns**

In [ ]:
def parameter_columns(dataframe):
    """
    function 
    selects parameter columns
    returns dataframe of parameters
    """
    param_cols = []
    for col in dataframe.columns:
        if 'Parameter' in col:
            param_cols.append(col)
#     print(param_cols)
    return dataframe[param_cols]

In [ ]:
# create new dataset with only the parameter columns
params = parameter_columns(df)

In [ ]:
# sort dataframe by date_time index 
params.sort_index(ascending = True, inplace=True)

In [ ]:
#visually quick_inspect data

params.plot(legend='right', xlim=('2005-07-01', '2022-12-01'), figsize=(20, 6));


**Note:** At first glance, it seems that 
- there was an uncharacteristic upsurge of TSS (Total Suspended Solids) in 2019 (or beginning of 2020) - wonder what could have happened
- level of TDS (Total Dissolved Solids flattened out after 2014)
- water tempreture seems seasonal - as expected

In [ ]:
# Mahdi's code

fig, axes = plt.subplots(figsize = (15, 20), ncols=2, nrows=8)

# flatten axes 
axes = axes.flatten()

for i in range(16):
    ax = axes[i]
    col_name = params.columns[i]
    ax.set_title(col_name)
    ax.set_xlabel("Date_Time")
    ax.set_ylabel(col_name)
    params[col_name].plot(ax = ax)
    
plt.tight_layout()

In [ ]:
# resample data to weekly periods
weekly_params = params.resample('W').mean()

In [ ]:
daily_params = params.resample('D').mean()

In [ ]:
params.shape

In [ ]:
weekly_params.shape

In [ ]:
daily_params.shape

In [ ]:
daily_params.dropna(inplace=True)

In [ ]:
weekly_params.columns

In [ ]:
fig, axes = plt.subplots(figsize = (15, 20), ncols=2, nrows=8)

# flatten axes 
axes = axes.flatten()

for i in range(16):
    ax = axes[i]
    col_name = weekly_params.columns[i]
    ax.set_title(col_name)
    ax.set_xlabel("Date_Time")
    ax.set_ylabel(col_name)
    weekly_params[col_name].plot(ax = ax)
    
plt.tight_layout()

In [ ]:
# plot daily data
fig, axes = plt.subplots(figsize = (15, 20), ncols=2, nrows=8)

# flatten axes 
axes = axes.flatten()

for i in range(16):
    ax = axes[i]
    col_name = daily_params.columns[i]
    ax.set_title(col_name)
    ax.set_xlabel("Date_Time")
    ax.set_ylabel(col_name)
    daily_params[col_name].plot(ax = ax)
    
plt.tight_layout()

In [ ]:
# # quick line plot
# for col in weekly_params.columns:
#     plt.figure(figsize=(20, 4))
#     plt.title(col)
#     weekly_params[col].plot();

#### Seasonal Decomposition of variables that show seasonality

In [ ]:
params.columns

In [ ]:
weekly_params = weekly_params.dropna()
weekly_params.isna().sum().sum()

In [ ]:

# weekly_trunc = params.loc[params.index > '2017-01-01']

In [ ]:
# create function to plot seasonal decomposition

def decompose(parameter, model, period):
    """
    function
    takes parameter as dataframe, model, period
    plot seasonal decomposition
    """
    decomp = seasonal_decompose(parameter, model=model, period=period)
    decomp.plot();

In [ ]:
# plot seasonal decomposition for each parameter in truncated dataframe (df) - weekly aggregate
# model = additive
# period = 52 (weekly seasonality) 

cols = weekly_params.columns
for col in cols:
    decompose(parameter=weekly_params[col],
              model='additive',
              period=52)
    
    # seasonality looks yearly

In [ ]:
# look at the standard deviation to eyeball whether seasonality is significant in any of the variables

weekly_params.describe()

In [ ]:
# plot seasonal decomposition for each parameter in truncated dataframe (df) - daily aggregate
# model = additive
# period = 7 (daily periods) 

cols = daily_params.columns
for col in cols:
    decompose(parameter=daily_params[col],
              model='additive',
              period=52)
    
    # seasonality looks yearly daily_params

**Autocorrelation Plots**

In [ ]:
# function to plot autocorrelation and partial autocorrelation

def autocorrelation(df, lags):
    plot_acf(df, lags=lags);
    plot_pacf(df, lags=lags);

In [ ]:
# plot autocorrelation for Clorophyll - weekly aggregates
print(cols[0])
autocorrelation(weekly_params[cols[0]], lags=60);

Clorophyll seems to have seasonality - roughly 1 year periods, no trend (the small lag values have relatively small, albiet positive, autocorrelation values).

In [ ]:
# plot autocorrelation for Dissolved Oxygen - weekly aggregates
print(cols[1])
autocorrelation(weekly_params[cols[1]], lags=60);

Dissolved oxygen - shows seasonality - roughly 1 year periods.
Trend: follows changes of physical seasons.

In [ ]:
# plot autocorrelation for Ammonia nitrogen - weekly aggregates
print(cols[2])
autocorrelation(weekly_params[cols[2]], lags=70);

Ammonia nitrogen - shows seasonal tendency - around 40 weeks, no trend(?).

In [ ]:
# plot autocorrelation for Nitrate Nitrogen - weekly aggregates
print(cols[3])
autocorrelation(weekly_params[cols[3]], lags=60);

Nitrate Nitrogen - shows seasonal changes (less than 1 year). No trend(?).

In [ ]:
# plot autocorrelation for PH level (corrected for temperature) - weekly aggregates
print(cols[4])
autocorrelation(weekly_params[cols[4]], lags=60);

PH level (corrected for temperature) - no seasonality, no trend. Lag 12 and 38 are the most important predictors.

In [ ]:
# plot autocorrelation for Orthophosphate Phosphorus - weekly aggregates
print(cols[5])
autocorrelation(weekly_params[cols[5]], lags=60);

Orthophosphate Phosphorus - no seasonality, no trend. Lag 45 is important.

In [ ]:
# plot autocorrelation for Salinity - weekly aggregates
print(cols[6])
autocorrelation(weekly_params[cols[6]], lags=60);

Salinity - not seasonal, no trend, lags 1, 3, and 4 seem important. 

In [ ]:
# plot autocorrelation for SECCHI DEPTH - weekly aggregates
print(cols[7])
autocorrelation(weekly_params[cols[7]], lags=60);

Secchi depth - no seasonality, no trend, lags 4, 32, 36 and 45 seem important. 

In [ ]:
# plot autocorrelation for Total Alkalinity - weekly aggregates
print(cols[8])
autocorrelation(weekly_params[cols[8]], lags=60);

Total Alkalinity - no seasonality (?), no trend, lag 2, 3, 4 and 47 seem important.

In [ ]:
# plot autocorrelation for Total Dissolved Solids - weekly aggregates
print(cols[9])
autocorrelation(weekly_params[cols[9]], lags=60);

Total Dissolved Solids - no seasonality, no trend, lag 1 and 43 seems important.

In [ ]:
# plot autocorrelation for Total Kjedahl Nitrogen - weekly aggregates
print(cols[10])
autocorrelation(weekly_params[cols[10]], lags=60);

Total Kjedahl Nitrogen - no seasonality, no trend.

In [ ]:
# plot autocorrelation for Total Nitrogen - weekly aggregates
print(cols[11])
autocorrelation(weekly_params[cols[11]], lags=60);

Total Nitrogen - shows seasonality (?), no trend, roughly yearly.

In [ ]:
# plot autocorrelation for Total Phosphorus - weekly aggregates
print(cols[12])
autocorrelation(weekly_params[cols[12]], lags=60);

Total Phosphorus - no seasonality, not trend, important lags: 14, 51.

In [ ]:
# plot autocorrelation for Total Suspended Solids - weekly aggregates
print(cols[13])
autocorrelation(weekly_params[cols[13]], lags=60);

Total suspended solids - no trend, no seasonality. 

In [ ]:
# plot autocorrelation for Turbidity - Nephelometric Method - weekly aggregates
print(cols[14])
autocorrelation(weekly_params[cols[14]], lags=60);

Water Clarity (turbidity, nephelometric method - used to test post filtration water clarity) - no seasonality, no trend. Important lags: 1, 5, 32, 38. <br>
source: https://www.hach.com/turbidity-article-turbidity101

In [ ]:
# plot autocorrelation for Water Temperature - weekly aggregates
print(cols[15])
autocorrelation(weekly_params[cols[15]], lags=60);

Water Temperature - shows seasonality (yearly), trend follows sine wave. Important lags: 1, 2, 5.

**Drop Parameter_TURB_NTU - and used only SECCHI as a metric for turbidity**

In [ ]:
weekly_df = weekly_params.drop(columns=['Parameter_TURB_NTU'])

In [ ]:
daily_df = daily_params.drop(columns=['Parameter_TURB_NTU'])

In [ ]:
daily_params.shape

In [ ]:
# make sure the data is in daily increments

daily_df.resample('D').mean().shape

In [ ]:
cols_df = weekly_df.columns

**Establish stationarity**<br>
Daily Aggregates

In [ ]:
# conduct Augmented Dicky-Fuller test - print test statistic and p-value

def interpret_adftest(target_column):
    '''Returns the Test Statistic and p-value for Augmented Dickey-Fuller test on given target column'''
    test = adfuller(target_column)
    output = pd.Series(test[0:2], index=['Test Statistic','p-value'])
    return output

# source: Mahdi

In [ ]:
# print out Dicky_Fuller test interpretations for each variable

for col in cols_df:
    test_stats = interpret_adftest(daily_df[col])
    print(test_stats)
    if test_stats[1] > 0.05:
        print(f'{col} not stationary')
    else:
        print(f'{col} stationary')
    print('')

All parameters are stationary.

**VAR Model**<br>
Small dataset ==> no train-test split for trainig the model. 

In [ ]:
params15 = params.drop(columns=['Parameter_TURB_NTU'])

In [ ]:
params15_monthly = params15.resample('M', closed='right', convention='end').mean()

In [ ]:
params15_monthly.isna().sum()

In [ ]:
params15_monthly.describe()

In [ ]:
# instantiate VAR model
var = VAR(params15_monthly)

In [ ]:
# store fitted model in variable fitted_var
# use Akaike information Criterion (aic)
# trend = 'c' ==> add constant


fitted_var = var.fit(maxlags=5, ic='aic', trend='c', verbose=True);

In [ ]:
fitted_var.summary()

In [ ]:
fitted_var.plot();


In [ ]:
# to evaluate the model = split the dataset into a 'training' and a 'testing' set (0.8 - 0.2 split)
split_point = int(params15_monthly.shape[0] * 0.8)
train = params15_monthly[:split_point]
test = params15_monthly[split_point:]

In [ ]:
train.head(2)

In [ ]:
train.tail(2)

In [ ]:
test.head(2)

In [ ]:
test.tail(2)

In [ ]:
# Mahdi's code
# get predictions and confidence intervals for "test" set
test_preds, lower_conf_ints, upper_conf_ints = fitted_var.forecast_interval(train.values, len(test))

In [ ]:
lower_conf_ints[:, 1]

In [ ]:
# Mahdi's code
# plot forecast
fig, axes = plt.subplots(figsize=(12,30), nrows = 15)

for i in range(15):
    col_name = cols[i]
    ax = axes[i]
    
    ax.set_title(col_name)
    
    train_col      = train[col_name] # pulling from pandas dataframe
    test_col       = test[col_name]  # pulling from pandas dataframe
    test_preds_col = test_preds[:,i] # pulling from numpy aray
    lower_conf_int = lower_conf_ints[:,i] # pulling from numpy aray
    upper_conf_int = upper_conf_ints[:,i] # pulling from numpy aray
    
    # Setting min and max (cuts off extreme conf. ints.)
    max_val = max([max(train_col), max(test_col)])
    min_val = min([min(train_col), min(test_col)])
    diff = max_val - min_val
    max_y = max_val + diff * 0.1
    min_y = min_val - diff * 0.1
    
    # setting min and max y limits (due to extreme conf. ints.)
    ax.set_ylim(ymin = min_y, ymax = max_y)
    
    # train data
    ax.plot(train.index, train_col, lw=1, color='green', ls='solid',label='Train Data')

    # test data
    ax.plot(test.index, test_col, lw=1, color='blue', ls='solid',label='Test Data')

    # forecast data
    ax.plot(test.index, test_preds_col, lw=1, color='red', ls="solid",  label='In-sample Predictions')
    ax.set_xlabel("Date")
    ax.set_ylabel(col_name)

    # conf ints
    ax.fill_between(test.index, lower_conf_int, upper_conf_int, color='k', alpha=0.1, label = "95% Conf. Int.");
    ax.legend(loc = "upper left")
    
plt.tight_layout();

**Note:** At this point the model does not predict well TP (total phosphorus). The in-data forecast approximates the test data well. However, I do not know how to interpret that both the forecast and the actual data points are outside the confidence interval. ==> __check what is going on__

In [ ]:
# Mahdi's code

# getting first column of predictions (Chlorophyll)
CHLA_preds = test_preds[:,0]
CHLA_true = test.iloc[:,0]
print('Clorophyll')
print('R2 score:', round(r2_score(CHLA_preds, CHLA_true), 2))




In [ ]:
# getting 13th column of predictions (Total Phosphorus)
TP_preds = test_preds[:,12]
TP_true = test.iloc[:,12]
print('Total Phosphorus')
print('R2 score:', round(r2_score(TP_preds, TP_true), 2))
